In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import statsmodels.api as sm
from sklearn.svm import LinearSVC, SVC, SVR
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, BaggingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('~/dsi/Submissions/Projects/project-5-group-project-master/kiva_numeric_models.csv')

In [4]:
df.head(2)

,ORIGINAL_LANGUAGE,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,...,DISTRIBUTION_MODEL,word_count_DT,word_count_LU,word_count_tags,char_count_DT,char_count_LU,char_count_tags,FEM_COUNT,MALE_COUNT,month
0,Spanish,The city of Portoviejo is located in the valle...,1075.0,1,1,0,Natural Medicines,Health,to purchase natural products.,EC,...,field_partner,192,4,5,924,26,36,1,1,1
1,English,"Lorna is a married woman, 39 years old with fi...",400.0,0,1,0,Personal Products Sales,Retail,"to purchase additional stocks of Avon, Natasha...",PH,...,field_partner,75,12,3,371,69,28,1,1,3


In [5]:
df.shape

(344588, 23)

In [6]:
df.isnull().sum()

ORIGINAL_LANGUAGE         0
DESCRIPTION_TRANSLATED    0
LOAN_AMOUNT               0
STATUS                    0
IMAGE_ID                  0
VIDEO_ID                  0
ACTIVITY_NAME             0
SECTOR_NAME               0
LOAN_USE                  0
COUNTRY_CODE              0
LENDER_TERM               0
TAGS                      0
REPAYMENT_INTERVAL        0
DISTRIBUTION_MODEL        0
word_count_DT             0
word_count_LU             0
word_count_tags           0
char_count_DT             0
char_count_LU             0
char_count_tags           0
FEM_COUNT                 0
MALE_COUNT                0
month                     0
dtype: int64

In [9]:
df.dtypes

ORIGINAL_LANGUAGE          object
DESCRIPTION_TRANSLATED     object
LOAN_AMOUNT               float64
STATUS                      int64
IMAGE_ID                    int64
VIDEO_ID                    int64
ACTIVITY_NAME              object
SECTOR_NAME                object
LOAN_USE                   object
COUNTRY_CODE               object
LENDER_TERM               float64
TAGS                       object
REPAYMENT_INTERVAL         object
DISTRIBUTION_MODEL         object
word_count_DT               int64
word_count_LU               int64
word_count_tags             int64
char_count_DT               int64
char_count_LU               int64
char_count_tags             int64
FEM_COUNT                   int64
MALE_COUNT                  int64
month                       int64
dtype: object

In [7]:
## Seperating catigorical features

cato_features = [feature for feature in df.columns if df[feature].dtypes=='object']
cato_features

['ORIGINAL_LANGUAGE',
 'DESCRIPTION_TRANSLATED',
 'ACTIVITY_NAME',
 'SECTOR_NAME',
 'LOAN_USE',
 'COUNTRY_CODE',
 'TAGS',
 'REPAYMENT_INTERVAL',
 'DISTRIBUTION_MODEL']

In [6]:
## Seperating numerical features

num_features = [feature for feature in df.columns if df[feature].dtypes!='object']
num_features

['LOAN_AMOUNT',
 'STATUS',
 'IMAGE_ID',
 'VIDEO_ID',
 'LENDER_TERM',
 'word_count_DT',
 'word_count_LU',
 'word_count_tags',
 'char_count_DT',
 'char_count_LU',
 'char_count_tags',
 'FEM_COUNT',
 'MALE_COUNT',
 'month']

In [5]:
# Dummies for some of the catigorical

df_dummies = pd.get_dummies(df, columns=['month','ORIGINAL_LANGUAGE','ACTIVITY_NAME','COUNTRY_CODE','REPAYMENT_INTERVAL','SECTOR_NAME','DISTRIBUTION_MODEL'],drop_first=True)
df_dummies.head(2)

,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,LOAN_USE,LENDER_TERM,TAGS,word_count_DT,word_count_LU,...,SECTOR_NAME_Food,SECTOR_NAME_Health,SECTOR_NAME_Housing,SECTOR_NAME_Manufacturing,SECTOR_NAME_Personal Use,SECTOR_NAME_Retail,SECTOR_NAME_Services,SECTOR_NAME_Transportation,SECTOR_NAME_Wholesale,DISTRIBUTION_MODEL_field_partner
0,The city of Portoviejo is located in the valle...,1075.0,1,1,0,to purchase natural products.,6.0,"#Repeat Borrower, #Health and Sanitation",192,4,...,0,1,0,0,0,0,0,0,0,1
1,"Lorna is a married woman, 39 years old with fi...",400.0,0,1,0,"to purchase additional stocks of Avon, Natasha...",14.0,"#Woman-Owned Business, #Parent",75,12,...,0,0,0,0,0,1,0,0,0,1


In [32]:
df['ACTIVITY_NAME'].value_counts()

Farming                      41986
General Store                27200
Personal Housing Expenses    21719
Food Production/Sales        13892
Retail                       12806
                             ...  
Celebrations                    10
Bookbinding                      8
Patchwork                        6
Film                             4
Adult Care                       3
Name: ACTIVITY_NAME, Length: 163, dtype: int64

In [8]:
# Small sample of df with dummies columns=['month','ORIGINAL_LANGUAGE','ACTIVITY_NAME','COUNTRY_CODE','REPAYMENT_INTERVAL',
# 'SECTOR_NAME','DISTRIBUTION_MODEL']

df_dummies_small = df_dummies.sample(200_000)
df_dummies_small.shape

(200000, 291)

In [9]:
# Small sample of numerical values

df_num_small = df[num_features].sample(200_000)
df_num_small.shape

(200000, 14)

`Model 1`
---
`LogReg parameters: Default`

`Data: df_num_small (200,000)`

In [3]:
X = df_num_small.drop(columns = ['STATUS'])
y = df_num_small['STATUS']

NameError: name 'df_num_small' is not defined

In [31]:
logreg = LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42, stratify=y)
logreg.fit(X_train,y_train)
print(f'train acc score: {logreg.score(X_train,y_train)}\ntest acc score: {logreg.score(X_test,y_test)}')

train acc score: 0.80804
test acc score: 0.80712


/Users/crivera/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


`Model 2`
---
`LogReg parameters: solver='liblinear', max_iter=500`

`Data: df_dummies_small (200,000)`


In [ ]:
## Liblinear tends to work better with small datasets

In [38]:
X = df_dummies_small.drop(columns = ['STATUS','DESCRIPTION_TRANSLATED','TAGS','LOAN_USE'])
y = df_dummies_small['STATUS']

In [41]:
logreg = LogisticRegression(solver='liblinear', max_iter=500)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42, stratify=y)
logreg.fit(X_train,y_train)
print(f'train acc score: {logreg.score(X_train,y_train)}\ntest acc score: {logreg.score(X_test,y_test)}')

train acc score: 0.8341266666666667
test acc score: 0.83406


`Model 3`
---
`Mutliple Regressors: All default parameters`

In [33]:
X = df_dummies_small.drop(columns = ['STATUS','DESCRIPTION_TRANSLATED','TAGS','LOAN_USE'])
y = df_dummies_small['STATUS']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42, stratify=y)

In [34]:
lr = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
bdt = BaggingClassifier()
rf = RandomForestClassifier()
adb = AdaBoostClassifier()
svr = SVC()

In [35]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [39]:
reg = [
#         LogisticRegression(),
#        KNeighborsClassifier(),
#        DecisionTreeClassifier(),
#        BaggingClassifier(),
       RandomForestClassifier()
#        AdaBoostClassifier(),
#        SVC()
]

In [40]:
def models(X_train,X_test,y_train,y_test,reg_list):   
    
    train_score=[]
    test_score=[]
    train_RMSE=[]
    test_RMSE=[]
    
    for reg in reg_list:
        
        reg.fit(X_train,y_train)
        train_score.append(reg.score(X_train,y_train))
        test_score.append(reg.score(X_test,y_test))
        train_pred = reg.predict(X_train)
        train_RMSE.append(mean_squared_error(y_train,train_pred))
        test_pred = reg.predict(X_test)
        test_RMSE.append(mean_squared_error(y_test,test_pred))
        
    return pd.DataFrame(data=[train_score, test_score, train_RMSE, test_RMSE],
                 index=['train_score', 'test_score','train_RMSE','test_RMSE'],
                 columns=[str(c) for c in reg_list]).T

In [41]:
models(X_train,X_test, y_train,y_test, reg_list)

,train_score,test_score,train_RMSE,test_RMSE
RandomForestRegressor(),0.912567,0.381234,0.013632,0.096472


In [42]:
df['ACTIVITY_NAME'].value_counts()

Farming                      41986
General Store                27200
Personal Housing Expenses    21719
Food Production/Sales        13892
Retail                       12806
                             ...  
Celebrations                    10
Bookbinding                      8
Patchwork                        6
Film                             4
Adult Care                       3
Name: ACTIVITY_NAME, Length: 163, dtype: int64

In [44]:
df[['ACTIVITY_NAME']].describe()

,ACTIVITY_NAME
count,344588
unique,163
top,Farming
freq,41986


In [47]:
df[['COUNTRY_CODE']].describe()

,COUNTRY_CODE
count,344588
unique,84
top,PH
freq,59281


In [7]:
df.loc[df['ACTIVITY_NAME']=='Celebrations',:]

,ORIGINAL_LANGUAGE,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,...,DISTRIBUTION_MODEL,word_count_DT,word_count_LU,word_count_tags,char_count_DT,char_count_LU,char_count_tags,FEM_COUNT,MALE_COUNT,month
27198,English,Abdeljabbar is a 28-year-old single Syrian ref...,1500.0,1,1,0,Celebrations,Personal Use,to pay for the expenses of his engagement cere...,LB,...,field_partner,106,9,5,482,43,59,0,1,1
59686,English,Wadhah is a hardworking young man who lives wi...,200.0,1,1,0,Celebrations,Personal Use,to purchase different foodstuffs for his famil...,YE,...,field_partner,215,18,1,1021,101,7,0,1,6
74292,English,Chanle is 22 years old and single. He is a gar...,1300.0,1,1,0,Celebrations,Personal Use,to pay for wedding celebrations for his family.,KH,...,field_partner,100,8,5,457,40,57,0,1,4
102918,English,Marina is 49 years old. She is a happy mother ...,1875.0,1,1,0,Celebrations,Personal Use,a divorced mother to organize a party for her ...,AL,...,field_partner,197,21,5,869,99,49,1,1,6
125185,English,Leonard is a 26-year-old man from the vulnerab...,1400.0,1,1,0,Celebrations,Personal Use,to help a member of the vulnerable Roma commun...,AL,...,field_partner,154,16,4,711,76,44,0,1,7
176139,English,"Kabibi is a married mother of one child, who a...",100.0,1,1,0,Celebrations,Personal Use,to purchase some foodstuffs for the Ramadan ce...,KE,...,field_partner,111,8,1,540,50,7,1,1,7
221851,English,Greetings from Huwaida! She is a 30-year-old m...,1025.0,1,1,0,Celebrations,Personal Use,"to prepare for the birth of her first child, w...",LB,...,field_partner,160,13,1,760,62,13,1,1,2
242755,English,Naomi is a single mother of 2 children and she...,100.0,1,1,0,Celebrations,Personal Use,to buy a goat for a celebration during the Eas...,KE,...,field_partner,108,14,4,475,61,30,1,1,5
277316,English,Raafat is a 30-year-old single Palestinian who...,1000.0,1,1,0,Celebrations,Personal Use,to pay for his engagement expenses.,LB,...,field_partner,91,6,3,416,30,41,0,1,0
338753,English,"Estela is 65 years old, married and a mother o...",800.0,1,1,0,Celebrations,Personal Use,"to pay for the upcoming wedding of her son, an...",PH,...,field_partner,145,19,3,692,82,24,1,1,1
